In [1]:
import pymysql
import sys
import requests
import json

connection = pymysql.connect(host = 'localhost',
                             user = 'root',
                             passwd = '',           #enter your mysql password before proceeding
                             charset='utf8', use_unicode=True);


In [2]:
numrecords = 10

In [3]:
##don't run this as part of start-up

#DROP TABLE

try: 
    cursor = connection.cursor()
    db_name = 'Yellow_Cab'
    table_name = 'Population'          #Replace this with any of the table name that exist so that you can drop it
    create_table_query = '''DROP TABLE {0}.{1} '''.format(db_name, table_name)
    cursor.execute(create_table_query)
    cursor.close()
except:
    print("could not drop 1")
    
##don't run this as part of start-up

#DROP TABLE

try:
    cursor = connection.cursor()
    db_name = 'Yellow_Cab'
    table_name = 'Trips'          #Replace this with any of the table name that exist so that you can drop it
    create_table_query = '''DROP TABLE {0}.{1} '''.format(db_name, table_name)
    cursor.execute(create_table_query)
    cursor.close()
except:
    print("could not drop 2")

In [4]:
# # Create a database

# db_name = 'Yellow_Cab'
# create_db_query = "CREATE DATABASE IF NOT EXISTS {0} DEFAULT CHARACTER SET 'utf8'".format(db_name)

# cursor = connection.cursor()
# cursor.execute(create_db_query)
# cursor.close()

In [4]:
##Create Population Table

cursor = connection.cursor()
db_name = 'Yellow_Cab'
table_name = 'Population'
# Create a table
# The {0} and {1} are placeholders for the parameters in the format(....) statement
create_table_query = '''CREATE TABLE IF NOT EXISTS {0}.{1} 
                                (info varchar(15),
                                PCT0120001 int, PCT0120002 int, 
                                P0170001 int,  
                                PRIMARY KEY(info)
                                )'''.format(db_name, table_name)

cursor.execute(create_table_query)
cursor.close()

In [5]:
##Create Trips Table

cursor = connection.cursor()
db_name = 'Yellow_Cab'
table_name = 'Trips'
# Create a table
# The {0} and {1} are placeholders for the parameters in the format(....) statement
create_table_query = '''CREATE TABLE IF NOT EXISTS {0}.{1} 
                                (trip_id int, 
                                passenger_count int,
                                trip_distance float(10),
                                pickup_longitude float(20),
                                pickup_latitude float(20),
                                RateCodeID int,
                                store_and_fwd_flag varchar(2),
                                dropoff_longitude float(20),
                                dropoff_latitude float(20),
                                Fare_amount float(10),
                                extra float(10),
                                mta_tax float(10),
                                tip_amount float(10),
                                tolls_amount float(10),
                                total_amount float(10),
                                vendor_id int,
                                pickup_datetime varchar(45),
                                dropoff_datetime varchar(45),
                                rate_code int,
                                payment_type int,
                                imp_surcharge float(10),
                                pickupinfo varchar(15),
                                dropoffinfo varchar(15),
                                PRIMARY KEY(trip_id)
                                )'''.format(db_name, table_name)

cursor.execute(create_table_query)
cursor.close()

In [6]:
## Get data from taxicab api

url = 'https://data.cityofnewyork.us/id/2yzn-sicd.json?$limit=%s' %(numrecords)
resp = requests.get(url)
results = json.loads(resp.text)

In [8]:
# check that results brought back the expected number of records (should == numrecords)

print(len(results))
print(results[0])

10
{'total_amount': '12.88', 'payment_type': '1', 'fare_amount': '8.5', 'passenger_count': '1', 'tip_amount': '2.58', 'rate_code': '1', 'extra': '1', 'mta_tax': '0.5', 'dropoff_longitude': '-73.97991943359375', 'tolls_amount': '0', 'store_and_fwd_flag': 'N', 'trip_distance': '1.23', 'pickup_latitude': '40.765678405761719', 'pickup_datetime': '2015-04-10T19:45:22.000', 'pickup_longitude': '-73.983688354492187', 'dropoff_datetime': '2015-04-10T19:56:34.000', 'vendor_id': '2', 'dropoff_latitude': '40.754909515380859'}


In [9]:
## insert data into Trips table



query_template = '''
INSERT INTO Yellow_Cab.Trips 
VALUES (%s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s,%s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s)'''

index=0
missing = 0

cursor = connection.cursor()
for entry in results:
    
    if index>numrecords:
        break

    try:

        trip_id = index
        
        passenger_count = entry["passenger_count"]
        
        trip_distance = entry["trip_distance"]
        pickup_longitude = entry["pickup_longitude"]
        pickup_latitude = entry["pickup_latitude"]
        try:
            ratecodeid = entry["ratecodeid"]
        except:
            ratecodeid = 0
        store_and_fwd_flag = entry["store_and_fwd_flag"]
        dropoff_longitude = entry["dropoff_longitude"]
        dropoff_latitude = entry["dropoff_latitude"]
        fare_amount = entry["fare_amount"]
        extra = entry["extra"]
        mta_tax = entry["mta_tax"]
        tip_amount = entry["tip_amount"]
        tolls_amount = entry["tolls_amount"]
        total_amount = entry["total_amount"]
        vendor_id = entry["vendor_id"]
        
        pickup_datetime = entry["pickup_datetime"]
        dropoff_datetime = entry["dropoff_datetime"]
    #   rate_code = entry["rate_code"]
        rate_code=0
        payment_type = entry["payment_type"]
        try:
            imp_surcharge = entry["imp_surcharge"]  
        except:
            imp_surcharge = 0
    #     print("Inserting trip details")
    #    print()



    #     url2 = "http://data.fcc.gov/api/block/find?format=json&latitude=%s&longitude=%s&showall=true" %(pickup_latitude,pickup_longitude)
    #     resp2 = requests.get(url2)
    #     results2 = json.loads(resp2.text)

    #     tract = results2["Block"]["FIPS"][5:11]
    #     print(results2)

    #     county = results2["Block"]["FIPS"][2:5]
    #     print(county)


    
    ## dropoff parameters
        dropoff_url2 = "http://data.fcc.gov/api/block/find?format=json&latitude=%s&longitude=%s&showall=true" %(dropoff_latitude,dropoff_longitude)
        dropoff_resp2 = requests.get(dropoff_url2)
        dropoff_results2 = json.loads(dropoff_resp2.text)

        dropoff_tract = dropoff_results2["Block"]["FIPS"][5:11]

        dropoff_county = dropoff_results2["Block"]["FIPS"][2:5]

        dropoffinfo =  dropoff_results2["Block"]["FIPS"][2:11]

    ## pickup parameters
        pickup_url2 = "http://data.fcc.gov/api/block/find?format=json&latitude=%s&longitude=%s&showall=true" %(pickup_latitude,pickup_longitude)
        pickup_resp2 = requests.get(pickup_url2)
        pickup_results2 = json.loads(pickup_resp2.text)

        pickup_tract = pickup_results2["Block"]["FIPS"][5:11]

        pickup_county = pickup_results2["Block"]["FIPS"][2:5]


        pickupinfo =  pickup_results2["Block"]["FIPS"][2:11]


        query_parameters = (trip_id, passenger_count, trip_distance, pickup_longitude, pickup_latitude, ratecodeid, store_and_fwd_flag, dropoff_longitude, dropoff_latitude, fare_amount, extra, mta_tax, tip_amount, tolls_amount, total_amount, vendor_id, pickup_datetime, dropoff_datetime, rate_code, payment_type, imp_surcharge, pickupinfo, dropoffinfo)

    #     query_parameters = (trip_id, passenger_count, trip_distance, pickup_longitude, pickup_latitude, ratecodeid, store_and_fwd_flag, dropoff_longitude, dropoff_latitude, fare_amount, extra, mta_tax, tip_amount, tolls_amount, total_amount, vendor_id, pickup_datetime, dropoff_datetime, rate_code, payment_type, imp_surcharge, tract,county)
        cursor.execute(query_template, query_parameters)

    except:
        missing = missing + 1
    index = index + 1    
print(missing)
connection.commit()
cursor.close()

0


In [10]:
#find distinct values of Tracts

import pandas as pd
cursor = connection.cursor()
db_name = 'Yellow_Cab'
table_name = 'Trips'          #Replace this with any of the table name that exist so that you can drop it
create_table_query = '''(Select distinct dropoffinfo from {0}.{1}) UNION (Select distinct pickupinfo from {0}.{1})'''.format(db_name, table_name)
cursor.execute(create_table_query)
response=cursor.fetchall()
cursor.close()
tractlist=(list(response))
print(tractlist)

## how do we append Select distinct pickup_tract??

df=pd.DataFrame(list(response))
df.head(6)

[('061009600',), ('061015400',), ('061014500',), ('061015801',), ('047004900',), ('061012700',), ('061012000',), ('061003800',), ('061007200',), ('061008100',), ('061013900',), ('061012600',), ('061013300',), ('081071600',), ('061009500',), ('061006300',), ('061009800',), ('061016300',)]


,0
0,061009600
1,061015400
2,061014500
3,061015801
4,047004900
5,061012700


In [11]:
## Insert Data into Population

query_template = '''
INSERT INTO Yellow_Cab.Population 
VALUES (%s, %s, %s, %s)'''

key="b0521cf638e6b5b2020031d032a0f0fad26d8678"


index=0
missing = 0

cursor = connection.cursor()
for a in range(0,numrecords,1):
    try:
    
#         print("a equals ",a)
#         print(len(tractlist))

        if index>numrecords*2:
            break
        if index>len(tractlist)-1:
            break


        #trip_id = index
    #     index=0

        county = tractlist[index][0][0:3]
        tract = tractlist[index][0][3:9]
#         print(county)
#         print(tract)
        info = tractlist[index][0]
#         print(info)


#         print("did we make it?")
        variablename = "PCT0120001"
        url3 = "http://api.census.gov/data/2010/sf1?get=%s&for=tract:%s&in=state:36&in=county:%s&key=b0521cf638e6b5b2020031d032a0f0fad26d8678" %(variablename, tract, county) 
        resp3 = requests.get(url3)
        results3 = json.loads(resp3.text)
        PCT0120001 = results3[1][0] 

        variablename = "PCT0120002"
        url3 = "http://api.census.gov/data/2010/sf1?get=%s&for=tract:%s&in=state:36&in=county:%s&key=b0521cf638e6b5b2020031d032a0f0fad26d8678" %(variablename, tract, county) 
        resp3 = requests.get(url3)
        results3 = json.loads(resp3.text)
        PCT0120002 = results3[1][0] 

        variablename = "P0170001"
        url3 = "http://api.census.gov/data/2010/sf1?get=%s&for=tract:%s&in=state:36&in=county:%s&key=b0521cf638e6b5b2020031d032a0f0fad26d8678" %(variablename, tract, county) 
        resp3 = requests.get(url3)
        results3 = json.loads(resp3.text)
        P0170001 = results3[1][0]

#         print('hi')

    #     what happened here??  we end up with an error and we miss the 6th one

        query_parameters = (info, PCT0120001, PCT0120002, P0170001)
        cursor.execute(query_template, query_parameters)


    except:
        missing = missing + 1
    index = index + 1


print(missing)

connection.commit()
cursor.close()

0


In [12]:
## make a dataframe with all data from Trips and Population for each cab ride


## 1st merge on dropoffinfo

cursor = connection.cursor(pymysql.cursors.DictCursor)

sql_query0 = '''
USE yellow_cab;
;'''

sql_query = '''
SELECT * from trips, population p2 where trips.dropoffinfo = p2.info;
;'''

cursor.execute(sql_query0)
cursor.execute(sql_query)
rows = cursor.fetchall()
df_from_sql = pd.DataFrame(list(rows))

## and rename the dropoffinfo columns so they're clear

df_from_sql = df_from_sql.rename(columns={'P0170001': 'Dropoff_AvgHhSize', 'PCT0120001': 'Dropoff_TotalPop',  'PCT0120002': 'Dropoff_MalePop'})
df_from_sql.head(6)
# print(df_from_sql)



,Fare_amount,Dropoff_AvgHhSize,Dropoff_TotalPop,Dropoff_MalePop,RateCodeID,dropoff_datetime,dropoff_latitude,dropoff_longitude,dropoffinfo,extra,...,pickup_longitude,pickupinfo,rate_code,store_and_fwd_flag,tip_amount,tolls_amount,total_amount,trip_distance,trip_id,vendor_id
0,8.5,1,155,88,0,2015-04-10T19:56:34.000,40.7549,-73.9799,061009600,1.0,...,-73.9837,061013900,0,N,2.58,0.0,12.88,1.23,0,2
1,7.5,2,13749,6127,0,2015-05-19T10:20:42.000,40.7812,-73.9493,061015400,0.0,...,-73.9646,061012000,0,N,0.00,0.0,8.30,1.89,1,2
2,14.5,2,5542,2576,0,2015-05-05T18:09:41.000,40.7683,-73.9824,061014500,1.0,...,-73.9569,061012600,0,N,3.25,0.0,19.55,1.80,2,1
3,14.0,2,5650,2545,0,2015-03-28T04:23:27.000,40.7829,-73.9518,061015801,0.5,...,-73.9886,061013300,0,N,2.00,0.0,17.30,3.80,3,1
4,54.5,2,2516,1140,0,2015-02-17T23:14:24.000,40.6910,-73.9973,047004900,0.5,...,-73.7768,081071600,0,N,11.16,0.0,66.96,20.39,4,2
5,10.5,2,6928,3872,0,2015-02-14T22:30:25.000,40.7613,-73.9896,061012700,0.5,...,-73.9967,061009500,0,N,0.00,0.0,11.80,1.10,5,1


In [13]:
## create a new df with just the data about the pickup tract

sql_query = '''
SELECT P0170001, PCT0120001, PCT0120002, dropoff_datetime, pickup_datetime, trip_ID from trips, population p2 where trips.pickupinfo = p2.info;
;'''

cursor.execute(sql_query)
rows = cursor.fetchall()
df_from_sql2 = pd.DataFrame(list(rows))

## and rename the pickupinfo columns so they're clear

df_from_sql2 = df_from_sql2.rename(columns={'P0170001': 'Pickup_AvgHhSize', 'PCT0120001': 'Pickup_TotalPop',  'PCT0120002': 'Pickup_MalePop', 'trip_ID': 'trip_ID2'})
df_from_sql2.head(6)

# test = pd.merge(df_from_sql, df_from_sql2, left_on="trip_ID", right_on="trip_ID2", how="outer")





,Pickup_AvgHhSize,Pickup_TotalPop,Pickup_MalePop,dropoff_datetime,pickup_datetime,trip_ID2
0,2,3971,1755,2015-05-19T10:20:42.000,2015-05-19T10:11:38.000,1
1,2,9237,4642,2015-01-03T00:53:41.000,2015-01-03T00:38:10.000,6


In [14]:
## finally merge on pickupinfo

df_from_sql =pd.merge(df_from_sql,df_from_sql2, left_on=["dropoff_datetime", "pickup_datetime"],right_on=["dropoff_datetime", "pickup_datetime"],how="left")
df_from_sql.head(6)

,Fare_amount,Dropoff_AvgHhSize,Dropoff_TotalPop,Dropoff_MalePop,RateCodeID,dropoff_datetime,dropoff_latitude,dropoff_longitude,dropoffinfo,extra,...,tip_amount,tolls_amount,total_amount,trip_distance,trip_id,vendor_id,Pickup_AvgHhSize,Pickup_TotalPop,Pickup_MalePop,trip_ID2
0,8.5,1,155,88,0,2015-04-10T19:56:34.000,40.7549,-73.9799,061009600,1.0,...,2.58,0.0,12.88,1.23,0,2,NaN,NaN,NaN,NaN
1,7.5,2,13749,6127,0,2015-05-19T10:20:42.000,40.7812,-73.9493,061015400,0.0,...,0.00,0.0,8.30,1.89,1,2,2.0,3971.0,1755.0,1.0
2,14.5,2,5542,2576,0,2015-05-05T18:09:41.000,40.7683,-73.9824,061014500,1.0,...,3.25,0.0,19.55,1.80,2,1,NaN,NaN,NaN,NaN
3,14.0,2,5650,2545,0,2015-03-28T04:23:27.000,40.7829,-73.9518,061015801,0.5,...,2.00,0.0,17.30,3.80,3,1,NaN,NaN,NaN,NaN
4,54.5,2,2516,1140,0,2015-02-17T23:14:24.000,40.6910,-73.9973,047004900,0.5,...,11.16,0.0,66.96,20.39,4,2,NaN,NaN,NaN,NaN
5,10.5,2,6928,3872,0,2015-02-14T22:30:25.000,40.7613,-73.9896,061012700,0.5,...,0.00,0.0,11.80,1.10,5,1,NaN,NaN,NaN,NaN
